In [8]:
import pandas as pd
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp

In [9]:
results = pd.read_csv('bunch_clustering_result_62_ver.csv')
results['extremity'] = 0
results['extremity_ground_truth'] = 0

In [12]:
results.head(300)

,name,version,bunch_algo,n_MoJo,mojoFM,extremity,extremity_ground_truth
0,apache-storm,1.1.0,ga,1632,20.583942,0.166667,0.003819
1,apache-storm,1.1.1,hillclimbing,1693,17.615572,0.791667,0.009393
2,apache-storm,1.1.2,hillclimbing,1224,40.437956,0.692308,0.016461
3,apache-storm,1.1.3,ga,1290,37.226277,0.333333,0.010352
4,apache-storm,1.1.4,ga,1495,27.250608,0.868852,0.105860
5,apache-storm,1.2.0,hillclimbing,1595,22.384428,0.757576,0.067227
6,apache-storm,1.2.1,hillclimbing,847,58.783455,0.714286,0.006742
7,apache-storm,1.2.2,exhaustive,1518,26.131387,0.631579,0.032328
8,apache-storm,1.2.3,hillclimbing,1195,41.849148,0.363636,0.016908
9,apache-storm,2.0.0,ga,2219,-7.980535,0.600000,0.011886


In [11]:
for index, element in results.iterrows():
    #print(index, element['name'])
    file_str = 'Final_Results_Bunch/' + element['name'] + '/' + element['name'] + '-' + element['version'] + '_' + element['bunch_algo'] + '_a.rsf'
    print(file_str)
    cluster_dict = {}
    with open(file_str, 'r') as f:
        lines = f.readlines()
        for line in lines:
            
            line = line.split(' ')
            parent = line[1]
            child = line[2]
            
            if parent in cluster_dict:
                cluster_dict[parent] += 1
            else:
                cluster_dict[parent] = 1
    
    total_clusters = 0
    extreme_clusters = 0
    for key in cluster_dict:
        total_clusters += 1
        if cluster_dict[key] < 5 or cluster_dict[key] > 100:
            extreme_clusters += 1
    
    final_extremity_score = 1 - (extreme_clusters/total_clusters)
    results.loc[(results.name == element['name']) & (results.version == element['version']), 'extremity'] = final_extremity_score
    
    file_str = 'Final_Results_Bunch/' + element['name'] + '/' + element['name'] + '-' + element['version'] + '_' + element['bunch_algo'] + '_b.rsf'
    print(file_str)
    cluster_dict = {}
    with open(file_str, 'r') as f:
        lines = f.readlines()
        for line in lines:
            
            line = line.split(' ')
            parent = line[1]
            child = line[2]
            
            if parent in cluster_dict:
                cluster_dict[parent] += 1
            else:
                cluster_dict[parent] = 1
    
    total_clusters = 0
    extreme_clusters = 0
    for key in cluster_dict:
        total_clusters += 1
        if cluster_dict[key] < 5 or cluster_dict[key] > 100:
            extreme_clusters += 1
    
    final_extremity_score = 1 - (extreme_clusters/total_clusters)
    results.loc[(results.name == element['name']) & (results.version == element['version']), 'extremity_ground_truth'] = final_extremity_score
            
        

Final_Results_Bunch/apache-storm/apache-storm-1.1.0_ga_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.0_ga_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.1_hillclimbing_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.1_hillclimbing_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.2_hillclimbing_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.2_hillclimbing_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.3_ga_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.3_ga_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.4_ga_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.1.4_ga_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.2.0_hillclimbing_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.2.0_hillclimbing_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.2.1_hillclimbing_a.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.2.1_hillclimbing_b.rsf
Final_Results_Bunch/apache-storm/apache-storm-1.2.2_exhaustive_a.r

Final_Results_Bunch/apache-maven/apache-maven-3.5.1_ga_b.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.2_ga_a.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.2_ga_b.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.3_exhaustive_a.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.3_exhaustive_b.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.4_exhaustive_a.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.5.4_exhaustive_b.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.6.0_ga_a.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.6.0_ga_b.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.6.1_ga_a.rsf
Final_Results_Bunch/apache-maven/apache-maven-3.6.1_ga_b.rsf
Final_Results_Bunch/apache-spark/apache-spark-2.2.3_hillclimbing_a.rsf
Final_Results_Bunch/apache-spark/apache-spark-2.2.3_hillclimbing_b.rsf
Final_Results_Bunch/apache-spark/apache-spark-2.3.0_hillclimbing_a.rsf
Final_Results_Bunch/apache-spark/apache-spark-2.3.0_hillclimbing_b.rsf
Final_Results

Final_Results_Bunch/bumptech-glide/bumptech-glide-4.6.0_exhaustive_a.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.6.0_exhaustive_b.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.6.1_ga_a.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.6.1_ga_b.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.7.0_hillclimbing_a.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.7.0_hillclimbing_b.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.7.1_hillclimbing_a.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.7.1_hillclimbing_b.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.8.0_hillclimbing_a.rsf
Final_Results_Bunch/bumptech-glide/bumptech-glide-4.8.0_hillclimbing_b.rsf
Final_Results_Bunch/codecentric-spring-boot-admin/codecentric-spring-boot-admin-2.1.0_hillclimbing_a.rsf
Final_Results_Bunch/codecentric-spring-boot-admin/codecentric-spring-boot-admin-2.1.0_hillclimbing_b.rsf
Final_Results_Bunch/codecentric-spring-boot-admin/codecentric-sp

Final_Results_Bunch/google-error-prone/google-error-prone-2.0.20_hillclimbing_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.0.21_exhaustive_a.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.0.21_exhaustive_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.0_hillclimbing_a.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.0_hillclimbing_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.1_exhaustive_a.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.1_exhaustive_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.2_hillclimbing_a.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.2_hillclimbing_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.3_ga_a.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.1.3_ga_b.rsf
Final_Results_Bunch/google-error-prone/google-error-prone-2.2.0_exhaustive_a.rsf
Final_Results_Bunch/google-erro

Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.7.1_ga_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.7.2_exhaustive_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.7.2_exhaustive_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.7.3_exhaustive_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.7.3_exhaustive_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.0_ga_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.0_ga_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.1_exhaustive_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.1_exhaustive_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.2_exhaustive_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.2_exhaustive_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.3_ga_a.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.3_ga_b.rsf
Final_Results_Bunch/naver-pinpoint/naver-pinpoint-1.8.4_hillclimbing_a.rsf
Final_Results_Bu

Final_Results_Bunch/hibernate-hibernate-orm/hibernate-hibernate-orm-5.4.4_hillclimbing_a.rsf
Final_Results_Bunch/hibernate-hibernate-orm/hibernate-hibernate-orm-5.4.4_hillclimbing_b.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.3.0_ga_a.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.3.0_ga_b.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.3.1_exhaustive_a.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.3.1_exhaustive_b.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.4.0_ga_a.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.4.0_ga_b.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.5.0_exhaustive_a.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.5.0_exhaustive_b.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.6.0_ga_a.rsf
Final_Results_Bunch/springfox-springfox/springfox-springfox-2.6.0_ga_b.rsf
Final_Results_Bunch/springfox-sp

Final_Results_Bunch/testcontainers-testcontainers-java/testcontainers-testcontainers-java-1.9.0_hillclimbing_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.1_ga_a.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.1_ga_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.2_exhaustive_a.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.2_exhaustive_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.3_ga_a.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.3_ga_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.4_exhaustive_a.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.4_exhaustive_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.5_hillclimbing_a.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.5_hillclimbing_b.rsf
Final_Results_Bunch/sofastack-sofa-rpc/sofastack-sofa-rpc-5.5.6_ga_a.rsf
Final_Results_Bunch/sofastack-

Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.5_ga_a.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.5_ga_b.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.6_ga_a.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.6_ga_b.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.7_exhaustive_a.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.7_exhaustive_b.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.8_ga_a.rsf
Final_Results_Bunch/oblac-jodd/oblac-jodd-5.0.8_ga_b.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.11_hillclimbing_a.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.11_hillclimbing_b.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.12_hillclimbing_a.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.12_hillclimbing_b.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.13_hillclimbing_a.rsf
Final_Results_Bunch/javaparser-javaparser/javaparser-javaparser-3.14.13_h

Final_Results_Bunch/pmd-pmd/pmd-pmd-6.4.0_exhaustive_a.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.4.0_exhaustive_b.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.5.0_exhaustive_a.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.5.0_exhaustive_b.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.6.0_hillclimbing_a.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.6.0_hillclimbing_b.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.7.0_ga_a.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.7.0_ga_b.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.8.0_hillclimbing_a.rsf
Final_Results_Bunch/pmd-pmd/pmd-pmd-6.8.0_hillclimbing_b.rsf
Final_Results_Bunch/ionic-team-capacitor/ionic-team-capacitor-0.0.110_hillclimbing_a.rsf
Final_Results_Bunch/ionic-team-capacitor/ionic-team-capacitor-0.0.110_hillclimbing_b.rsf
Final_Results_Bunch/ionic-team-capacitor/ionic-team-capacitor-0.0.111_exhaustive_a.rsf
Final_Results_Bunch/ionic-team-capacitor/ionic-team-capacitor-0.0.111_exhaustive_b.rsf
Final_Results_Bunch/ionic-team-capacitor/ionic-team-capacitor-0.0.

In [14]:
results.to_csv('bunch_clustering_result_extremity_mojo_75_ver.csv', index=False)

In [15]:
results.sort_values('extremity_ground_truth', ascending=False)

,name,version,bunch_algo,n_MoJo,mojoFM,extremity,extremity_ground_truth
144,dropwizard-dropwizard,1.3.5,exhaustive,636,47.740345,0.812500,0.823529
147,dropwizard-dropwizard,1.3.8,hillclimbing,629,48.315530,0.857143,0.769231
496,cglib-cglib,3.2.8,hillclimbing,148,40.322581,0.900000,0.769231
235,mockito-mockito,3.0.6,hillclimbing,627,41.511194,0.866667,0.764706
493,ionic-team-capacitor,0.0.111,exhaustive,13,98.389095,1.000000,0.750000
228,mockito-mockito,2.28.2,hillclimbing,583,45.615672,0.758621,0.722222
230,mockito-mockito,3.0.1,exhaustive,617,42.444030,0.925926,0.685714
506,raphw-byte-buddy,1.9.7,exhaustive,817,32.255390,0.842105,0.681818
497,cglib-cglib,3.2.9,ga,97,60.887097,0.857143,0.666667
436,iSoron-uhabits,1.7.8,hillclimbing,212,67.131783,0.800000,0.666667


In [6]:
with open('Final_Results_Bunch/activiti-activiti/activiti-activiti-7.1.74_exhaustive_b.rsf', 'r') as f:
    cluster_dict = {}
    lines = f.readlines()
    for line in lines:

        line = line.split(' ')
        parent = line[1]
        child = line[2]

        if parent in cluster_dict:
            cluster_dict[parent] += 1
        else:
            cluster_dict[parent] = 1

    total_clusters = 0
    extreme_clusters = 0
    for key in cluster_dict:
        total_clusters += 1
        if cluster_dict[key] < 5 or cluster_dict[key] > 100:
            extreme_clusters += 1
    
    final_extremity_score = 1 - (extreme_clusters/total_clusters)
    print(final_extremity_score)

0.6216216216216216
